In [2]:
import pandas as pd
import sys
sys.path.append('./utils')
from utils import *
from rank import *
from process import * 
from tqdm.notebook import tqdm
import seaborn as sns

# データ保存先
data_folda = '../data/'

* masterからデータを追加

In [54]:

### master data から加えるデータを定義
add_list=['name',"prefecture_name",'head_branch','opened_on', 'created_at','genre_first_name', 
          'genre_second_name','genre_third_name','northlatitude', 'eastlongitude',
          'net_reservation_flg','price_range_lunch_owner','price_range_dinner_owner', 
          'price_range_lunch_user','price_range_dinner_user',
          # 20240312追加
          'all_photo_count','photo_food_count','photo_drink_count',
          'photo_interior_count','photo_exterior_count','official_info_flg','max_visit_month']

# make_timeseries_data.ipynb で作成したデータを読み込む
pseudo_time_series=pd.read_pickle(data_folda+'skeleton_train.pkl')

# 直近三カ月の閉店率を計算して保存
# pseudo_time_series=pseudo_time_series[pseudo_time_series['base_date']>='2022-08-31']
# close_ratios=calculate_close_ratio(pseudo_time_series, data_folda)
# close_ratios=pd.read_pickle(data_folda+'close_ratios.pkl')

# 三カ月の閉店率を読み込み
close_ratios=pd.read_pickle(data_folda+'close_ratios.pkl')

# 2023-01-01からの閉店を予測する。base_date が 2023-01-01の一日前以後が対象
pseudo_time_series=pseudo_time_series[pseudo_time_series['base_date']>='2022-12-31']
# base_date が 2023年1月1日以降のデータを抽出 (closed_date がそれ以降の閉店店舗しかクロールしていないため)
pseudo_time_series = add_master_data(pseudo_time_series, data_folda, add_list, "restaurant_data.csv")
pseudo_time_series=pd.merge(pseudo_time_series,close_ratios,on=["base_date","genre_first_name"],how="left")
pseudo_time_series.to_pickle(data_folda+'train_with_master.pkl')

pseudo_time_series_test3=pd.read_pickle(data_folda+'skeleton_test_3month.pkl')
pseudo_time_series_test3 = add_master_data(pseudo_time_series_test3, data_folda, add_list, "restaurant_data.csv")
pseudo_time_series_test3=pd.merge(pseudo_time_series_test3,close_ratios,on=["base_date","genre_first_name"],how="left")
pseudo_time_series_test3.to_pickle(data_folda+'skeleton_test_3month_master.pkl')

pseudo_time_series_test6=pd.read_pickle(data_folda+'skeleton_test_6month.pkl')
pseudo_time_series_test6 = add_master_data(pseudo_time_series_test6, data_folda, add_list, "restaurant_data.csv")
pseudo_time_series_test3=pd.merge(pseudo_time_series_test6,close_ratios,on=["base_date","genre_first_name"],how="left")
pseudo_time_series_test6.to_pickle(data_folda+'skeleton_test_6month_master.pkl')

* V_GOOGLE からデータを追加

In [13]:
pseudo_time_series = pd.read_pickle(data_folda+'train_with_master.pkl')
add_list=["restaurant_id","overview.aggregateRating","overview.reviewCount","general_rank"]
pseudo_time_series = add_v_google(pseudo_time_series, data_folda, add_list)
pseudo_time_series.to_pickle(data_folda+'train_with_master_google.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_3month_master.pkl')
add_list=["restaurant_id","overview.aggregateRating","overview.reviewCount","general_rank"]
pseudo_time_series_test = add_v_google(pseudo_time_series_test, data_folda, add_list)
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_3month_master_google.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_6month_master.pkl')
add_list=["restaurant_id","overview.aggregateRating","overview.reviewCount","general_rank"]
pseudo_time_series_test = add_v_google(pseudo_time_series_test, data_folda, add_list)
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_6month_master_google.pkl')

del pseudo_time_series, pseudo_time_series_test

/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

* V_RETTY からデータを追加

In [3]:
#追加する変数をリストで定義
add_list=["restaurant_id",'infos.isOfficial', 
       'infos.familiar.users.count', 'infos.familiar.stars',
       'general_rank_RETTY', 
       'infos.onlineReservation_RETTY', 'infos.access.transferTime1','infos.access.transferTime2','infos.access.transferTime3',
       'infos.numberOfSeats.value', 'infos.counterSeats.value',
       'infos.privateRoom.value', 'infos.updateInfo.firstReviewDate',
       'infos.wantToGo', 'infos.updateInfo.lastReviewDate',
       'infos.went', 'infos.rateByWent', 'infos.excellent', 'infos.coupon',
       'infos.photoCount.Photograph', 'infos.photoCount.Cooking',
       'infos.photoCount.Interior', 'infos.photoCount.Exterior',
       'infos.photoCount.Menu', 'infos.reviewCount.Review',
       'infos.reviewCount.Lunch', 'infos.reviewCount.Dinner',
       'menus.course_list_count',
       'menus.dishes_list_count', 'menus.lunch_list_count',
       'menus.drink_list_count', 'menus.takeout_list_count',
       'infos.familiar.users.genge','infos.reserveEntireStore']
 
pseudo_time_series = pd.read_pickle(data_folda+'train_with_master_google.pkl')         
pseudo_time_series = add_v_retty(pseudo_time_series, data_folda, add_list, master_name="V_RETTY",read_from_snowflake=False)
pseudo_time_series.to_pickle(data_folda+'train_with_master_google_retty.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_3month_master_google.pkl')
pseudo_time_series_test = add_v_retty(pseudo_time_series_test, data_folda, add_list, master_name="V_RETTY",read_from_snowflake=False)
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_3month_master_google_retty.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_6month_master_google.pkl')
pseudo_time_series_test = add_v_retty(pseudo_time_series_test, data_folda, add_list, master_name="V_RETTY",read_from_snowflake=False)
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_6month_master_google_retty.pkl')

del pseudo_time_series, pseudo_time_series_test

/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

* HOTPEPPER からデータを追加

In [3]:
add_list=['restaurant_id','general_rank_HOTPEPPER','infos.aggregateRating','infos.ratingReview',
          'infos.satisfaction.percentage1','infos.satisfaction.percentage2','infos.satisfaction.percentage3',
          'infos.couponCount','infos.onlineReservation_HOTPEPPER','infos.ownerRegistration','infos.ticket',
          'photos.allCount','photos.appearanceCount','photos.drinkCount','photos.foodCount','photos.otherCount',
          'photos.postCount','menus.courseListCount']


pseudo_time_series = pd.read_pickle(data_folda+'train_with_master_google_retty.pkl')
pseudo_time_series=pseudo_time_series[pseudo_time_series['base_date']>='2022-12-31']
pseudo_time_series= reduce_mem_usage(pseudo_time_series)
pseudo_time_series = add_v_hotpepper(pseudo_time_series, data_folda, add_list, master_name="V_HOTPEPPER",read_from_snowflake=False)
pseudo_time_series.to_pickle(data_folda+'train_with_master_google_retty_hotpepper.pkl')

pseudo_time_series_test3 = pd.read_pickle(data_folda+'skeleton_test_3month_master_google_retty.pkl')
pseudo_time_series_test3= reduce_mem_usage(pseudo_time_series_test3)
pseudo_time_series_test3 = add_v_hotpepper(pseudo_time_series_test3, data_folda, add_list, master_name="V_HOTPEPPER",read_from_snowflake=False) 
pseudo_time_series_test3.to_pickle(data_folda+'skeleton_test_3month_master_google_retty_hotpepper.pkl')

pseudo_time_series_test6 = pd.read_pickle(data_folda+'skeleton_test_6month_master_google_retty.pkl')
pseudo_time_series_test6= reduce_mem_usage(pseudo_time_series_test6)
pseudo_time_series_test6 = add_v_hotpepper(pseudo_time_series_test6, data_folda, add_list, master_name="V_HOTPEPPER",read_from_snowflake=False) 
pseudo_time_series_test6.to_pickle(data_folda+'skeleton_test_6month_master_google_retty_hotpepper.pkl')

del pseudo_time_series, pseudo_time_series_test3, pseudo_time_series_test6

/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

* V_STRUCTURED からデータを取得

In [4]:
add_list = '"restaurant_id","url","name_rank","address_rank","telephone_rank","aggregateRating.ratingCount", \
    "aggregateRating.ratingValue", "aggregateRating.reviewCount", \
    "aggregateRating.bestRating", "aggregateRating.worstRating", "interactionCount.went", \
    "interactionCount.wanttogo","paymentAccepted"'


pseudo_time_series = pd.read_pickle(data_folda+'train_with_master_google_retty_hotpepper.pkl')
pseudo_time_series = add_v_structured(pseudo_time_series, data_folda, add_list, master_name="V_STRUCTURED",read_from_snowflake=True)
pseudo_time_series.to_pickle(data_folda+'train_with_master_google_retty_hotpepper_structured.pkl')

del pseudo_time_series

pseudo_time_series_test3 = pd.read_pickle(data_folda+'skeleton_test_3month_master_google_retty_hotpepper.pkl')
pseudo_time_series_test3 = add_v_structured(pseudo_time_series_test3, data_folda, add_list, master_name="V_STRUCTURED",read_from_snowflake=True) 
pseudo_time_series_test3.to_pickle(data_folda+'skeleton_test_3month_master_google_retty_hotpepper_structured.pkl')

pseudo_time_series_test6 = pd.read_pickle(data_folda+'skeleton_test_6month_master_google_retty_hotpepper.pkl')
pseudo_time_series_test6 = add_v_structured(pseudo_time_series_test6, data_folda, add_list, master_name="V_STRUCTURED",read_from_snowflake=True) 
pseudo_time_series_test6.to_pickle(data_folda+'skeleton_test_6month_master_google_retty_hotpepper_structured.pkl')

del pseudo_time_series_test3, pseudo_time_series_test6

select "restaurant_id","url","name_rank","address_rank","telephone_rank","aggregateRating.ratingCount",     "aggregateRating.ratingValue", "aggregateRating.reviewCount",     "aggregateRating.bestRating", "aggregateRating.worstRating", "interactionCount.went",     "interactionCount.wanttogo","paymentAccepted" from KRAKEN_DG.KRAKEN_USER.V_STRUCTURED


/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

select "name","hostname" from KRAKEN_DG.KRAKEN_USER.V_MEDIA_SITE_LIST
demae-can.com
twitter.com
www.google.co.jp
www.tripadvisor.jp
sarah30.com
paypaygourmet.yahoo.co.jp
hitosara.com
place.line.me
www.ozmall.co.jp
4travel.jp
rtrp.jp
www.instagram.com
www.ekiten.jp
www.navitime.co.jp
www.jalan.net
r.gnavi.co.jp
www.facebook.com
retty.me
restaurant.ikyu.com
tabelog.com
www.ubereats.com
gourmet.aumo.jp
loco.yahoo.co.jp
autoreserve.com
www.hotpepper.jp
ramendb.supleks.jp
tokyo-calendar.jp
select "restaurant_id","url","name_rank","address_rank","telephone_rank","aggregateRating.ratingCount",     "aggregateRating.ratingValue", "aggregateRating.reviewCount",     "aggregateRating.bestRating", "aggregateRating.worstRating", "interactionCount.went",     "interactionCount.wanttogo","paymentAccepted" from KRAKEN_DG.KRAKEN_USER.V_STRUCTURED


/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

select "name","hostname" from KRAKEN_DG.KRAKEN_USER.V_MEDIA_SITE_LIST
demae-can.com
twitter.com
www.google.co.jp
www.tripadvisor.jp
sarah30.com
paypaygourmet.yahoo.co.jp
hitosara.com
place.line.me
www.ozmall.co.jp
4travel.jp
rtrp.jp
www.instagram.com
www.ekiten.jp
www.navitime.co.jp
www.jalan.net
r.gnavi.co.jp
www.facebook.com
retty.me
restaurant.ikyu.com
tabelog.com
www.ubereats.com
gourmet.aumo.jp
loco.yahoo.co.jp
autoreserve.com
www.hotpepper.jp
ramendb.supleks.jp
tokyo-calendar.jp
select "restaurant_id","url","name_rank","address_rank","telephone_rank","aggregateRating.ratingCount",     "aggregateRating.ratingValue", "aggregateRating.reviewCount",     "aggregateRating.bestRating", "aggregateRating.worstRating", "interactionCount.went",     "interactionCount.wanttogo","paymentAccepted" from KRAKEN_DG.KRAKEN_USER.V_STRUCTURED


/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

select "name","hostname" from KRAKEN_DG.KRAKEN_USER.V_MEDIA_SITE_LIST
demae-can.com
twitter.com
www.google.co.jp
www.tripadvisor.jp
sarah30.com
paypaygourmet.yahoo.co.jp
hitosara.com
place.line.me
www.ozmall.co.jp
4travel.jp
rtrp.jp
www.instagram.com
www.ekiten.jp
www.navitime.co.jp
www.jalan.net
r.gnavi.co.jp
www.facebook.com
retty.me
restaurant.ikyu.com
tabelog.com
www.ubereats.com
gourmet.aumo.jp
loco.yahoo.co.jp
autoreserve.com
www.hotpepper.jp
ramendb.supleks.jp
tokyo-calendar.jp


* V_TRIPADVISOR から特徴量を追加

In [3]:
add_list=["restaurant_id",'general_rank_TRIPADVISOR','infos.aggregateRating_TRIPADVISOR',"infos.reviewCount","infos.qa_count"]

pseudo_time_series = pd.read_pickle(data_folda+'train_with_master_google_retty_hotpepper_structured.pkl')
pseudo_time_series = reduce_mem_usage(pseudo_time_series)
pseudo_time_series = add_v_tripadvisor(pseudo_time_series, data_folda, add_list, master_name="V_TRIPADVISOR",read_from_snowflake=False)
pseudo_time_series.to_pickle(data_folda+'train_with_master_google_retty_hotpepper_structured_tripadvisor.pkl')

del pseudo_time_series

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_3month_master_google_retty_hotpepper_structured.pkl')
pseudo_time_series_test = reduce_mem_usage(pseudo_time_series_test)
pseudo_time_series_test = add_v_tripadvisor(pseudo_time_series_test, data_folda, add_list, master_name="V_TRIPADVISOR",read_from_snowflake=False) 
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_3month_master_google_retty_hotpepper_structured_tripadvisor.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_6month_master_google_retty_hotpepper_structured.pkl')
pseudo_time_series_test = reduce_mem_usage(pseudo_time_series_test)
pseudo_time_series_test = add_v_tripadvisor(pseudo_time_series_test, data_folda, add_list, master_name="V_TRIPADVISOR",read_from_snowflake=False) 
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_6month_master_google_retty_hotpepper_structured_tripadvisor.pkl')

del pseudo_time_series_test

/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

* V_HITOSARA から特徴量を追加

In [4]:
# hitosara premium を追加

add_list=["restaurant_id","general_rank_HITOSARA","infos.premium","infos.onlineReservation_HITOSARA"]

pseudo_time_series = pd.read_pickle(data_folda+'train_with_master_google_retty_hotpepper_structured_tripadvisor.pkl')
pseudo_time_series= reduce_mem_usage(pseudo_time_series)
pseudo_time_series = add_v_hitosara(pseudo_time_series, data_folda, add_list, master_name="V_HITOSARA",read_from_snowflake=False)
pseudo_time_series.to_pickle(data_folda+'train_with_master_structured_main_medias.pkl')

del pseudo_time_series

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_3month_master_google_retty_hotpepper_structured_tripadvisor.pkl')
pseudo_time_series_test= reduce_mem_usage(pseudo_time_series_test)
pseudo_time_series_test = add_v_hitosara(pseudo_time_series_test, data_folda, add_list, master_name="V_HITOSARA",read_from_snowflake=False) 
pseudo_time_series_test.to_pickle(data_folda+'test3_with_master_structured_main_medias.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_6month_master_google_retty_hotpepper_structured_tripadvisor.pkl')
pseudo_time_series_test= reduce_mem_usage(pseudo_time_series_test)
pseudo_time_series_test = add_v_hitosara(pseudo_time_series_test, data_folda, add_list, master_name="V_HITOSARA",read_from_snowflake=False) 
pseudo_time_series_test.to_pickle(data_folda+'test6_with_master_structured_main_medias.pkl')

del pseudo_time_series_test

/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

In [4]:
pseudo_time_series=pd.read_pickle(data_folda+'train_with_master_structured_main_medias.pkl')
pseudo_time_series.info()
pseudo_time_series_filter=pseudo_time_series.drop_duplicates(subset=['restaurant_id'])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10503768 entries, 0 to 10503767
Columns: 114 entries, restaurant_id to infos.onlineReservation_HITOSARA
dtypes: bool(1), category(3), datetime64[ns](5), float32(47), float64(4), int16(3), int32(5), int8(4), object(42)
memory usage: 6.2+ GB


In [5]:
pseudo_time_series_filter

,restaurant_id,original_id,closed_date,base_date,target3,target6,eval_period,id,name,prefecture_name,...,paymentAccepted,general_rank_STRUCTURED,hostname,general_rank_TRIPADVISOR,infos.aggregateRating_TRIPADVISOR,infos.reviewCount,infos.qa_count,general_rank_HITOSARA,infos.premium,infos.onlineReservation_HITOSARA
0,00000528-ea7b-4364-bb76-3d46fba520ef,22001886,NaT,2022-12-31,0,0,True,22001886,ハッピーバレー,静岡県,...,NaN,NaN,NaN,S,3.5,2,0,S,0.0,0.0
12,00002990-5cde-49f2-a2e1-baca20c61fdc,27007296,2023-02-22,2022-12-31,1,1,True,27007296,印度屋,大阪府,...,NaN,NaN,NaN,A,4,14,0,NaN,NaN,NaN
24,00003d54-0a2a-43f5-b945-5ae9b0314942,22035496,NaT,2022-12-31,0,0,True,22035496,セブンイレブン,静岡県,...,,S,食べログ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,00003ef6-4867-409a-896c-566134666192,22013184,NaT,2022-12-31,0,0,True,22013184,久福鰻店,静岡県,...,,S,食べログ,L,5,4,0,NaN,NaN,NaN
48,00005256-746d-467d-bf65-82686b244108,23061742,NaT,2022-12-31,0,0,True,23061742,天下一品,愛知県,...,NaN,NaN,NaN,S,2.5,3,0,S,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10503708,ffff88ea-fd34-46f8-a749-001a2c59cdba,22022016,NaT,2022-12-31,0,0,True,22022016,京会席 銀明翠,静岡県,...,NaN,NaN,NaN,L,5,3,0,S,0.0,0.0
10503720,ffff9349-bd6a-4c56-bb06-c1c11af3b736,11036290,NaT,2022-12-31,0,0,True,11036290,雪みるく,埼玉県,...,NaN,NaN,NaN,N,,,,A,0.0,0.0
10503732,ffffbb36-5a31-4f02-b1e5-5883dfc16f09,40011715,NaT,2022-12-31,0,0,True,40011715,ビッグアップ,福岡県,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,0.0,0.0
10503744,ffffdb1a-05bf-4227-8ea8-cf5bb0b63a43,47003447,NaT,2022-12-31,0,0,True,47003447,信濃路,沖縄県,...,NaN,NaN,NaN,L,,,0,NaN,NaN,NaN


In [6]:
# pseudo_time_series_filterからカラム名に general_rank もの含むを抽出
pseudo_time_series_filter_columns = pseudo_time_series_filter.columns
general_rank_columns = [column for column in pseudo_time_series_filter_columns if 'general_rank' in column]
general_rank_columns

['general_rank_GOOGLE',
 'general_rank_RETTY',
 'general_rank_HOTPEPPER',
 'general_rank_STRUCTURED',
 'general_rank_TRIPADVISOR',
 'general_rank_HITOSARA']

In [ ]:
# pseudo_time_series_filter[general_rank_columns]をプロット
# 横軸を S,A,B,L,F,N,NaNの分布
# 一つのキャンバスにプロット

fig, ax = plt.subplots()

for column in general_rank_columns:
    print(column,pseudo_time_series_filter[column].unique())
    # 前のプロットを上書きする
    ax.hist(pseudo_time_series_filter[column])

plt.show()
"""
for column in general_rank_columns:
    print(column,pseudo_time_series_filter[column].unique())
    # 前のプロットを上書きする
    pseudo_time_series_filter[column].value_counts().plot(kind='bar',title=column,ax=plt.gca())
    plt.show()
"""


In [41]:
skeleton_train["general_rank_HITOSARA"]

0           S
1           S
2           S
3           S
4           S
           ..
52518835    S
52518836    S
52518837    S
52518838    S
52518839    S
Name: general_rank_HITOSARA, Length: 52518840, dtype: object

In [11]:
skeleton_train=pd.read_pickle(data_folda+'skeleton_train.pkl')
skeleton_train=pd.merge(skeleton_train,pseudo_time_series_filter[["restaurant_id"]+general_rank_columns],on="restaurant_id",how="left")
skeleton_train.to_pickle(data_folda+'skeleton_train_with_general_rank.pkl')

In [3]:
skeleton_test=pd.read_pickle(data_folda+'skeleton_test.pkl')
skeleton_test=pd.merge(skeleton_test,pseudo_time_series_filter[["restaurant_id"]+general_rank_columns],on="restaurant_id",how="left")
skeleton_test.to_pickle(data_folda+'skeleton_test_with_general_rank.pkl')

NameError: name 'pseudo_time_series_filter' is not defined

In [134]:
pseudo_time_series_filter[pseudo_time_series_filter["general_rank_HITOSARA"].isin(["A","B","S","L","N","F"])]["general_rank_HITOSARA"].value_counts()

S    546677
A     50424
L     34853
B      1130
F        25
N         1
Name: general_rank_HITOSARA, dtype: int64

In [55]:
g_filter=pseudo_time_series_filter[pseudo_time_series_filter["general_rank_GOOGLE"].isin(["A","B","S","L","N","F"])]
g_filter_=g_filter[g_filter["general_rank_RETTY"].isin(["A","B","S","L","N","F"])]
g_filter_=g_filter_[g_filter_["general_rank_HOTPEPPER"].isin(["A","B","S","L","N","F"])]
g_filter_=g_filter_[g_filter_["general_rank_HITOSARA"].isin(["A","B","S","L","N","F"])]
g_filter_=g_filter_[g_filter_["general_rank_TRIPADVISOR"].isin(["A","B","S","L","N","F"])]
g_filter_=g_filter_[g_filter_["general_rank_STRUCTURED"].isin(["A","B","S","L","N","F"])]

print(g_filter.describe())


        original_id        target3        target6            id  \
count  8.694380e+05  869438.000000  869438.000000  8.694380e+05   
mean   2.083029e+07       0.007363       0.013388  2.083029e+07   
std    1.199797e+07       0.085494       0.114929  1.199797e+07   
min    1.000001e+06       0.000000       0.000000  1.000001e+06   
25%    1.303462e+07       0.000000       0.000000  1.303462e+07   
50%    2.000145e+07       0.000000       0.000000  2.000145e+07   
75%    2.801496e+07       0.000000       0.000000  2.801496e+07   
max    4.703118e+07       1.000000       1.000000  4.703118e+07   

       northlatitude  eastlongitude  net_reservation_flg  \
count  868633.000000  868633.000000        869438.000000   
mean       35.507799     137.054747             0.082691   
std         2.559043       3.995540             0.275415   
min         0.000000       0.000000             0.000000   
25%        34.673365     135.416475             0.000000   
50%        35.437477     138.207717 

In [136]:
pseudo_time_series_filter[pseudo_time_series_filter["general_rank_GOOGLE"].isin(["A","B","S","L","N","F"])]["general_rank_HITOSARA"].value_counts()

S    544765
A     50302
L     34550
B      1130
F        25
N         1
Name: general_rank_HITOSARA, dtype: int64

In [ ]:
ggoogle=pseudo_time_series_filter[pseudo_time_series_filter["general_rank_GOOGLE"].isin(["A","B","S","L","N","F"])]["general_rank_GOOGLE"].value_counts()

In [8]:
rank_num=pd.DataFrame()
for column in general_rank_columns:
    print(column,pseudo_time_series_filter[column].dropna().shape[0])
    #上記を転置してrank_numに追加
    rank_num=rank_num.append([[column,pseudo_time_series_filter[column].dropna().shape[0]]])

general_rank_GOOGLE 869438
general_rank_RETTY 637495


/tmp/ipykernel_7303/3118007309.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rank_num=rank_num.append([[column,pseudo_time_series_filter[column].dropna().shape[0]]])


general_rank_HOTPEPPER 517620
general_rank_STRUCTURED 294864


/tmp/ipykernel_7303/3118007309.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rank_num=rank_num.append([[column,pseudo_time_series_filter[column].dropna().shape[0]]])
/tmp/ipykernel_7303/3118007309.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rank_num=rank_num.append([[column,pseudo_time_series_filter[column].dropna().shape[0]]])


general_rank_TRIPADVISOR 413841
general_rank_HITOSARA 633110


/tmp/ipykernel_7303/3118007309.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rank_num=rank_num.append([[column,pseudo_time_series_filter[column].dropna().shape[0]]])


In [131]:
rank_num.sort_values(by=1,ascending=False)

,0,1
0,general_rank_GOOGLE,869438
0,general_rank_RETTY,637495
0,general_rank_HITOSARA,633110
0,general_rank_HOTPEPPER,517620
0,general_rank_TRIPADVISOR,413841
0,general_rank_STRUCTURED,294864


In [48]:
rank_num=pd.DataFrame()
for column in general_rank_columns:
    num=pseudo_time_series_filter[pseudo_time_series_filter["closed_date"].isna()==False][column].dropna().shape[0]
    ratio=num/len(pseudo_time_series_filter[pseudo_time_series_filter["closed_date"].isna()==False])
    print(column,num,ratio)
    #上記を転置してrank_numに追加
    rank_num=rank_num.append([[column,num,ratio]])
rank_num

general_rank_GOOGLE 18153 0.9909383699983624


/tmp/ipykernel_7303/1519989912.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rank_num=rank_num.append([[column,num,ratio]])


general_rank_RETTY 13491 0.7364484960969485
general_rank_HOTPEPPER 9424 0.5144385610568263
general_rank_STRUCTURED 10434 0.5695725749222119


/tmp/ipykernel_7303/1519989912.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rank_num=rank_num.append([[column,num,ratio]])


general_rank_TRIPADVISOR 8679 0.4737704023145368


/tmp/ipykernel_7303/1519989912.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rank_num=rank_num.append([[column,num,ratio]])


general_rank_HITOSARA 2532 0.13821715159124406


/tmp/ipykernel_7303/1519989912.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rank_num=rank_num.append([[column,num,ratio]])


In [51]:
rank_num=pd.DataFrame()
for column in general_rank_columns:
    num=pseudo_time_series_filter[column].dropna().shape[0]
    ratio=num/len(pseudo_time_series_filter)
    print(column,num,ratio)
    #上記を転置してrank_numに追加
    rank_num=rank_num.append([[column,num,ratio]])
rank_num

general_rank_GOOGLE 869438 0.9932869804435894
general_rank_RETTY 637495 0.7283043570650075
general_rank_HOTPEPPER 517620 0.5913535028572604
general_rank_STRUCTURED 294864 0.3368665416067834


/tmp/ipykernel_7303/1641020988.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rank_num=rank_num.append([[column,num,ratio]])
/tmp/ipykernel_7303/1641020988.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rank_num=rank_num.append([[column,num,ratio]])


general_rank_TRIPADVISOR 413841 0.4727914782580879
general_rank_HITOSARA 633110 0.7232947262353853


/tmp/ipykernel_7303/1641020988.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rank_num=rank_num.append([[column,num,ratio]])
/tmp/ipykernel_7303/1641020988.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rank_num=rank_num.append([[column,num,ratio]])


,0,1,2
0,general_rank_GOOGLE,869438,0.993287
0,general_rank_RETTY,637495,0.728304
0,general_rank_HOTPEPPER,517620,0.591354
0,general_rank_STRUCTURED,294864,0.336867
0,general_rank_TRIPADVISOR,413841,0.472791
0,general_rank_HITOSARA,633110,0.723295


In [35]:
pseudo_time_series_filter["closed_date"].dropna().shape[0]

18319

In [45]:
rank_num.sort_values(by=1,ascending=False)

,0,1
0,general_rank_GOOGLE,18153
0,general_rank_RETTY,13491
0,general_rank_STRUCTURED,10434
0,general_rank_HOTPEPPER,9424
0,general_rank_TRIPADVISOR,8679
0,general_rank_HITOSARA,2532


In [10]:
g_filter=pseudo_time_series_filter
rlist=["A","B","S","L","N","F"]
rlist=["A","B","S"]
#num_restaurants=pd.DataFrame()
for column in rank_num.sort_values(by=1,ascending=False)[0].values:
    if(column=="general_rank_STRUCTURED"):
        continue
    g_filter=g_filter[g_filter[column].isin(rlist)]
    num_restaurants=num_restaurants.append([[column,g_filter[g_filter[column].isin(rlist)][column].shape[0]]])

/tmp/ipykernel_7303/543598693.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  num_restaurants=num_restaurants.append([[column,g_filter[g_filter[column].isin(rlist)][column].shape[0]]])
/tmp/ipykernel_7303/543598693.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  num_restaurants=num_restaurants.append([[column,g_filter[g_filter[column].isin(rlist)][column].shape[0]]])
/tmp/ipykernel_7303/543598693.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  num_restaurants=num_restaurants.append([[column,g_filter[g_filter[column].isin(rlist)][column].shape[0]]])
/tmp/ipykernel_7303/543598693.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instea

In [29]:
# general_rank_GOOGLE, general_rank_RETTY, general_rank_HOTPEPPER, general_rank_HITOSARA, general_rank_TRIPADVISORそれぞれがある無しで、どの程度のレストランが抽出されるかを確認

# 各メディアの true False を保存する dataframe を作成
num_restaurants=pd.DataFrame()

#rlist=["A","B","S","L","N","F"]
rlist=["A","B","S"]

for is_google in [True,False]:
    for is_retty in [True,False]:
        for is_hotpepper in [True,False]:
            for is_hitosara in [True,False]:
                for is_tripadvisor in [True,False]:
                    g_filter=pseudo_time_series_filter
                    if is_google:
                        g_filter=g_filter[g_filter["general_rank_GOOGLE"].isin(rlist)]
                    if is_retty:
                        g_filter=g_filter[g_filter["general_rank_RETTY"].isin(rlist)]
                    if is_hotpepper:
                        g_filter=g_filter[g_filter["general_rank_HOTPEPPER"].isin(rlist)]
                    if is_hitosara:
                        g_filter=g_filter[g_filter["general_rank_HITOSARA"].isin(rlist)]
                    if is_tripadvisor:
                        g_filter=g_filter[g_filter["general_rank_TRIPADVISOR"].isin(rlist)]
                    print(is_google,is_retty,is_hotpepper,is_hitosara,is_tripadvisor,g_filter.shape[0])
                    
                    num_restaurants=num_restaurants.append([[is_google,is_retty,is_hotpepper,is_hitosara,is_tripadvisor,g_filter.shape[0]]])

True True True True True 81058


/tmp/ipykernel_7303/498613659.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  num_restaurants=num_restaurants.append([[is_google,is_retty,is_hotpepper,is_hitosara,is_tripadvisor,g_filter.shape[0]]])


True True True True False 166415
True True True False True 88447
True True True False False 191736


/tmp/ipykernel_7303/498613659.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  num_restaurants=num_restaurants.append([[is_google,is_retty,is_hotpepper,is_hitosara,is_tripadvisor,g_filter.shape[0]]])


True True False True True 118365


...

/tmp/ipykernel_7303/498613659.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  num_restaurants=num_restaurants.append([[is_google,is_retty,is_hotpepper,is_hitosara,is_tripadvisor,g_filter.shape[0]]])


False False False True False 598231


/tmp/ipykernel_7303/498613659.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  num_restaurants=num_restaurants.append([[is_google,is_retty,is_hotpepper,is_hitosara,is_tripadvisor,g_filter.shape[0]]])
/tmp/ipykernel_7303/498613659.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  num_restaurants=num_restaurants.append([[is_google,is_retty,is_hotpepper,is_hitosara,is_tripadvisor,g_filter.shape[0]]])


False False False False True 199718
False False False False False 875314


/tmp/ipykernel_7303/498613659.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  num_restaurants=num_restaurants.append([[is_google,is_retty,is_hotpepper,is_hitosara,is_tripadvisor,g_filter.shape[0]]])


In [30]:
#num_restaurants　の is_google, is_retty, is_hotpepper, is_hitosara, is_tripadvisor を0,1,2,3,4から代入
num_restaurants.columns=["is_google","is_retty","is_hotpepper","is_hitosara","is_tripadvisor","num_restaurants"]

In [27]:
num_restaurants

,is_google,is_retty,is_hotpepper,is_hitosara,is_tripadvisor,num_restaurants
0,True,True,True,True,True,251776
0,True,True,True,True,False,366712
0,True,True,True,False,True,287423
0,True,True,True,False,False,435798
0,True,True,False,True,True,311464
0,True,True,False,True,False,512371
0,True,True,False,False,True,361836
0,True,True,False,False,False,634156
0,True,False,True,True,True,269153
0,True,False,True,True,False,411548


In [31]:
#num_restaurants=num_restaurants.drop([0,1,2,3,4,5],axis=1)
num_restaurants["num_restaurants"]=num_restaurants["num_restaurants"].astype(int)
num_restaurants.sort_values(by="num_restaurants",ascending=False,inplace=True)
num_restaurants[["is_google","is_retty","is_hotpepper","is_hitosara","is_tripadvisor","num_restaurants"]]
num_restaurants


,is_google,is_retty,is_hotpepper,is_hitosara,is_tripadvisor,num_restaurants
0,False,False,False,False,False,875314
0,False,False,False,True,False,598231
0,True,False,False,False,False,573939
0,False,True,False,False,False,451540
0,True,False,False,True,False,418996
0,False,True,False,True,False,359697
0,True,True,False,False,False,354547
0,False,False,True,False,False,302134
0,True,True,False,True,False,291669
0,True,False,True,False,False,242427
